In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import numpy as np
import pandas as pd
import tqdm

from james.msmarco.ms_marco_eval import load_reference, compute_metrics_james_zhao

In [3]:
residing_folder = "./data/results/baseline"
knn_index_file = "knn.npy"
output_file = "inference.tsv"

In [4]:
top1000_dev_ids = pd.read_csv("./data/top1000/top1000.dev.ids.tsv", header=None, sep="\t", names=["qid", "pid"])
top1000_dev_inputs = pd.read_csv("./data/top1000/top1000.dev.inputs.tsv", header=None, sep="\t", names=["qid", "query"])
knn_results = np.load(os.path.join(residing_folder, knn_index_file))

In [5]:
print(f"{len(top1000_dev_ids)=}, {len(top1000_dev_inputs)=}, {len(knn_results)=}")

len(top1000_dev_ids)=6668967, len(top1000_dev_inputs)=6980, len(knn_results)=6980


In [6]:
# qid to ranked ids
inference = {}
for qid, knn_row in zip(top1000_dev_inputs["qid"], knn_results):
  inference[qid]= knn_row.tolist()

In [29]:
qrels = pd.read_csv("./data/qrels/qrels.dev.tsv", header=None, sep="\t", names=["qid", "iteration_num", "pid", "relevance"])
reference = {}
for qid, pid in qrels[["qid", "pid"]].iterrows():
  if qid not in reference:
    reference[qid] = []
  if pid in reference[qid]:
    print(f"ERROR, {qid=}, {pid=}")
  reference[qid].append(pid)

In [22]:
qid0 = top1000_dev_inputs["qid"][0]
first_result = knn_results[0][0]
reference0 = reference[qid0]

In [24]:
scores, ranking = compute_metrics_james_zhao(
  qids_to_relevant_passageids=reference,
  qids_to_ranked_candidate_passages=inference,
  max_mrr_rank=10,
)

In [25]:
from collections import Counter
print(Counter(ranking))

Counter({1: 5886, 2: 479, 3: 182, 0: 171, 4: 99, 6: 46, 5: 45, 7: 26, 8: 21, 9: 14, 10: 11})


In [26]:
print(scores)

{'MRR @10': 0.8934925751580456, 'QueriesRanked': 6980}
